# Rapport du TP2

Importation des dépendances liées aux TP:

In [23]:
from sklearn import datasets, model_selection
from sklearn.metrics import precision_score
import numpy as np
from sklearn.neural_network import MLPClassifier

- Charger le jeu de données MNIST

In [24]:
mnist = datasets.fetch_mldata('MNIST original')

- Diviser la base de données en 49000 lignes pour l’apprentissage (training) et le reste pour les tests

In [25]:
index = np.random.randint(70000, size=49000)
data = mnist.data[index]
target = mnist.target[index]

- Construire un modèle de classification ayant comme paramètre: `hidden_layer_sizes = (50)`, puis calculez la précession du classifieur 

In [26]:
# Construire modèle
MLP_model = MLPClassifier(hidden_layer_sizes=(50))

# Choisir données d'apprentissage et du test
x_train, x_test, y_train, y_test = model_selection.train_test_split(data, target, train_size=0.8)

# Apprendre et mesurer précesion
MLP_model.fit(x_train, y_train)
y_predict = MLP_model.predict(x_test)
precision_score = precision_score(y_test, y_predict, average='micro')

# Afficher score
print("Precision = ", precision_score)

/usr/local/insa/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Precision =  0.9594897959183674


- Afficher la classe de l’image 4 et sa classe prédite

In [27]:
image_4 = xtest[4].reshape(1,-1)
print("Valeur prédite : ", MLP_model.predict(image_4))
print("===========================================")
print("L'image: ")
plt.imshow(xtest.reshape((-1, 28, 28))[4],cmap=plt.cm.gray_r,interpolation="nearest")
plt.show()

NameError: name 'xtest' is not defined